In [7]:
import chardet
import pandas as pd
import os
import matplotlib.pyplot as plt
import seaborn as sns
from tabulate import tabulate

def detectar_codificacao(file_path, n_bytes=10000):
    with open(file_path, 'rb') as f:
        rawdata = f.read(n_bytes)
    result = chardet.detect(rawdata)
    return result['encoding']

def ler_csv_com_melhor_separador(file_path):
    encoding = detectar_codificacao(file_path)
    try:
        df = pd.read_csv(file_path, sep=';', encoding=encoding)
    except:
        df = pd.read_csv(file_path, sep=',', encoding=encoding)
    return df

def limpar_e_padronizar(df, nome_arquivo):
    df.columns = df.columns.str.strip().str.lower()
    
    if "senador" in nome_arquivo:
        df.rename(columns={"senador": "nome", "valor": "valor_liquidado"}, inplace=True)

    if 'valor_liquidado' not in df.columns:
        possiveis = [col for col in df.columns if "valor" in col]
        if possiveis:
            df.rename(columns={possiveis[0]: "valor_liquidado"}, inplace=True)
    
    df = df[df["valor_liquidado"].notna()]
    df["valor_liquidado"] = (
        df["valor_liquidado"]
        .astype(str)
        .str.replace(",", ".")
        .str.replace("R\$", "")
        .str.replace(" ", "")
        .astype(float)
    )
    
    if 'nome' not in df.columns:
        possiveis = [col for col in df.columns if "senador" in col or "parlamentar" in col]
        if possiveis:
            df.rename(columns={possiveis[0]: "nome"}, inplace=True)

    df["nome"] = df["nome"].str.strip().str.title()
    return df[["nome", "valor_liquidado"]]

def agrupar_e_analisar(df):
    agrupado = df.groupby("nome")["valor_liquidado"].agg(["sum", "mean", "std", "count"]).reset_index()
    agrupado = agrupado.sort_values("sum", ascending=False)

    print("\nResumo por Senador:")
    print(tabulate(agrupado.head(10), headers="keys", showindex=False, floatfmt=".2f"))

    print("\nEstatísticas Gerais:")
    print(df["valor_liquidado"].describe())

    return agrupado

def visualizar_dados(agrupado):
    plt.figure(figsize=(12, 6))
    sns.barplot(data=agrupado.head(10), x="sum", y="nome", palette="viridis")
    plt.title("Top 10 Senadores por Gasto Total")
    plt.xlabel("Gasto Total (R$)")
    plt.ylabel("Senador")
    plt.tight_layout()
    plt.show()

    plt.figure(figsize=(10, 4))
    sns.boxplot(x=agrupado["sum"])
    plt.title("Distribuição dos Gastos Totais por Senador")
    plt.xlabel("Gasto Total (R$)")
    plt.tight_layout()
    plt.show()

def main():
    base_dir = os.path.dirname(os.path.abspath(__file__))
    arquivos = ["despesas_institucional.csv", "gastos_ceaps_2022.csv"]
    
    dfs = []
    for arquivo in arquivos:
        path = os.path.join(base_dir, arquivo)
        if not os.path.exists(path):
            print(f"Arquivo não encontrado: {arquivo}")
            continue
        print(f"\nLendo: {arquivo}")
        df = ler_csv_com_melhor_separador(path)
        df_limpo = limpar_e_padronizar(df, arquivo)
        dfs.append(df_limpo)

    if dfs:
        df_total = pd.concat(dfs, ignore_index=True)
        agrupado = agrupar_e_analisar(df_total)
        visualizar_dados(agrupado)
    else:
        print("Nenhum dado válido encontrado.")

if __name__ == "__main__":
    main()

<>:38: SyntaxWarning: invalid escape sequence '\$'
<>:38: SyntaxWarning: invalid escape sequence '\$'
C:\Users\User\AppData\Local\Temp\ipykernel_21108\2031528207.py:38: SyntaxWarning: invalid escape sequence '\$'
  .str.replace("R\$", "")


NameError: name '__file__' is not defined